## Write a Web Service

Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output).

The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X 


In [1]:
%%time

import pandas as pd
from datetime import datetime

bookings_file = 'bookings.csv.bz2'
data = pd.read_csv(bookings_file, sep = '^', chunksize = 1000000)

partial_results = []

for chunk_num, chunk in enumerate(data):
    
    print('Start processing chunk %d at %s' % (chunk_num, datetime.now()))
    
    chunk = chunk[['arr_port', 'pax', 'year']].dropna()
    chunk = chunk[chunk['year'] == 2013]
    result_from_this_chunk = chunk.groupby('arr_port')['pax'].sum()
    
    partial_results.append(result_from_this_chunk)
    
all_results = pd.concat(partial_results)
total_arrivals = all_results.groupby('arr_port').sum()
sorted_by_arrivals = total_arrivals.sort_values(ascending = False)
sorted_by_arrivals.index = sorted_by_arrivals.index.str.strip()

Start processing chunk 0 at 2018-06-02 13:31:09.755007
Start processing chunk 1 at 2018-06-02 13:31:34.166281
Start processing chunk 2 at 2018-06-02 13:31:58.904248
Start processing chunk 3 at 2018-06-02 13:32:24.047505
Start processing chunk 4 at 2018-06-02 13:32:48.765895


<string>:2: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.


Start processing chunk 5 at 2018-06-02 13:33:13.735675
Start processing chunk 6 at 2018-06-02 13:33:38.879540
Start processing chunk 7 at 2018-06-02 13:34:03.644247
Start processing chunk 8 at 2018-06-02 13:34:31.826140
Start processing chunk 9 at 2018-06-02 13:34:58.524032
Start processing chunk 10 at 2018-06-02 13:34:58.732454
CPU times: user 3min 53s, sys: 14.2 s, total: 4min 8s
Wall time: 4min 13s


In [9]:
from flask import Flask, request
from werkzeug.serving import run_simple


app = Flask('topnairports-service')

@app.route('/topnairports')
def topnairports():
    
    n = int(request.args.get('n'))
    return sorted_by_arrivals.head(n).to_json()
                      
run_simple('localhost', 5000, app)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jun/2018 13:42:01] "GET /topnairports?n=5 HTTP/1.1" 200 -


http://localhost:5000/topnairports?n=5

## Web Service

http://flask.pocoo.org/docs/1.0/quickstart

http://127.0.0.1:5000/hello